In [1]:
# %%capture
# !pip install -U datasets
# !pip install -U trl
# !pip install -U transformers accelerate
# !pip install -U bitsandbytes
# # # # # uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from dotenv import load_dotenv
import os
from peft import LoraConfig
from peft import TaskType
from trl import SFTConfig, SFTTrainer
from transformers import BitsAndBytesConfig
# load_dotenv()

from huggingface_hub import login
login()

print("torch version: ", torch.__version__)
print('is gpu enbled: ', torch.cuda.is_available())

torch version:  2.6.0+cu124
is gpu enbled:  True


In [3]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [4]:
# Load model and tokenizer
model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  quantization_config=bnb_config,
                                                  device_map='auto')
# meta-llama/Llama-3.2-3B-Instruct

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# for name, weights in model.named_parameters():
#     print(name)
#     print(weights)
#     break

In [6]:
# Load the dataset
from datasets import load_dataset
ds = load_dataset("Jofthomas/hermes-function-calling-thinking-V1")

In [7]:
ds['train'][0]

{'conversations': [{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title'

In [8]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,)

In [9]:
# ds['train'][12] # sample data

In [10]:
sample_data = ds['train'][0]['conversations']

In [11]:
# we won't use bos token in here , since it's will be added when we set SFTTrainer and data get tokenized, so we set new chat template there with bos token and no other difference
chat_template_for_preprocessing = "{% if  messages[0]['role']==system%}{{ raise_exception('System message is not supported in gemma, it would be good to merget the system prompt with first user message')}}{% endif %}{% for message in messages %}{{'<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<eos_turn><eos>\n'}}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"
chat_template = "{{ bos_token }}{% if  messages[0]['role']==system%}{{ raise_exception('System message is not supported in gemma, it would be good to merget the system prompt with first user message')}}{% endif %}{% for message in messages %}{{'<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<eos_turn><eos>\n'}}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"


In [12]:
tokenizer.chat_template = chat_template_for_preprocessing

In [13]:
print(tokenizer.apply_chat_template(sample_data, tokenize=False))

<start_of_turn>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Argumen

In [14]:
ds = ds.rename_column('conversations', 'messages')

In [15]:
# preprocess dataset {apply chat template}

def preprocess_dataset(row):
    messages = row['messages'] # Dict['str', List[Dict['str', 'str']]]

    # Check if there system message, if yes, merge the system prompt with first user input, since the gemma model does not have system prompt , it trained in this way
    if messages[0]['role'] == 'system':
        system_message = messages[0]['content']
        messages[1]['content'] = system_message + "Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thought}</think>\n\n" + messages[1]['content']
        # after merging, remove the system message from message
        messages.pop(0)

    return {'text': tokenizer.apply_chat_template(messages, tokenize=False)}


In [16]:
ds = ds.map(preprocess_dataset, remove_columns=['messages'])

In [17]:
ds = ds['train'].train_test_split(.1, seed=0)

In [18]:
test_split = ds['test'].train_test_split(0.4, seed=0)

ds['validation'] = test_split['train']
ds['test'] = test_split['test']

In [19]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3213
    })
    test: Dataset({
        features: ['text'],
        num_rows: 143
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 214
    })
})

In [20]:
from dataclasses import dataclass
from enum import Enum

class ChatToolLMSpecialTokens(Enum):
    tools = '<tools>'
    eos_tools = '</tools>'
    think = '<think>'
    eos_think = '</think>'
    tool_response = '<tool_response>'
    eos_tool_response = '</tool_response>'
    pad_token = '<pad>'
    eos_token = '<eos>'

    @classmethod
    def special_token_list(clss):
        return [cls_token.value for cls_token in clss]

ChatToolLMSpecialTokens.special_token_list()


['<tools>',
 '</tools>',
 '<think>',
 '</think>',
 '<tool_response>',
 '</tool_response>',
 '<pad>',
 '<eos>']

In [21]:
list(ChatToolLMSpecialTokens)

[<ChatToolLMSpecialTokens.tools: '<tools>'>,
 <ChatToolLMSpecialTokens.eos_tools: '</tools>'>,
 <ChatToolLMSpecialTokens.think: '<think>'>,
 <ChatToolLMSpecialTokens.eos_think: '</think>'>,
 <ChatToolLMSpecialTokens.tool_response: '<tool_response>'>,
 <ChatToolLMSpecialTokens.eos_tool_response: '</tool_response>'>,
 <ChatToolLMSpecialTokens.pad_token: '<pad>'>,
 <ChatToolLMSpecialTokens.eos_token: '<eos>'>]

In [22]:
ChatToolLMSpecialTokens.pad_token.value

'<pad>'

In [23]:
# Load tokenizer again so we can add special tokens easly
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          pad_token=ChatToolLMSpecialTokens.pad_token.value,
                                          eos_token=ChatToolLMSpecialTokens.eos_token.value,
                                          additional_special_tokens=ChatToolLMSpecialTokens.special_token_list())

In [24]:
# set new chat template with bos token for sft auto data processing
tokenizer.chat_template = chat_template

In [25]:
peft_config = LoraConfig(r=64,
                         lora_alpha=128,
                         target_modules=['q_proj', 'v_proj', 'o_proj', 'embed_tokens', 'k_proj', 'lm_head'], # 'gate_proj', 'up_proj', 'down_proj'],
                         task_type=TaskType.CAUSAL_LM,
                         lora_dropout=0.01)

In [26]:
len(tokenizer) # Total number of tokens

128264

In [27]:
# since we added some special tokenzer , we need to add some random embedding for it
base_model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128264, 3072)

In [29]:
from peft import get_peft_model, prepare_model_for_kbit_training

base_model.gradient_checkpointing_enable()

peft_model = prepare_model_for_kbit_training(base_model)
peft_model = get_peft_model(peft_model, peft_config=peft_config)

peft_model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:574: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens', 'lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [30]:
peft_model.print_trainable_parameters()

trainable params: 53,511,168 || all params: 3,266,285,568 || trainable%: 1.6383


In [31]:
training_arugment = SFTConfig(output_dir='gemma_function_calling_and_thinking',
                              per_device_train_batch_size=1,
                              per_device_eval_batch_size=1,
                              gradient_accumulation_steps=4,
                              optim="paged_adamw_8bit",
                              logging_first_step=True,
                              logging_dir='runs',
                              learning_rate=1e-4,
                              max_grad_norm=1.0,
                              num_train_epochs=2,
                              warmup_ratio=0.1,
                              lr_scheduler_type='cosine',
                              eval_strategy='steps',
                              save_strategy='steps',
                              report_to='tensorboard',
                              gradient_checkpointing=True,
                              # gradient_checkpointing_kwargs={"use_reentrant": False},
                              packing=False,
                              save_steps=100,
                              eval_steps=10,
                              logging_steps=10,
                              fp16=True,
                              max_seq_length=1500)

In [32]:
trainer =  SFTTrainer(model=peft_model,
                      processing_class=tokenizer,
                      peft_config=peft_config,
                      train_dataset=ds['train'],
                      eval_dataset=ds['validation'].select(range(50)),
                      args=training_arugment)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:
peft_model.device

device(type='cuda', index=0)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
10,1.762600,1.775071
20,1.675200,1.608636
30,1.517600,1.396928
40,1.287800,1.107835
50,1.003600,0.817067
60,0.807000,0.702693
70,0.707800,0.652285
80,0.650000,0.580116
90,0.543500,0.463054
100,0.454100,0.401399


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


## Infernce

In [ ]:
dataloader = trainer.get_train_dataloader()

In [ ]:
for i in dataloader:
    break

In [ ]:
tokenizer.decode(tokenizer.all_special_ids)

In [ ]:
print(tokenizer.batch_decode(i['input_ids'])[0])

In [ ]:
inp = tokenizer.batch_decode(i['input_ids'])[0]
labels = tokenizer.batch_decode(i['labels'])[0]
inp==labels

In [ ]:
# len(i['labels'][0])

In [ ]:
# print(labels)

In [ ]:
# tokenizer.decode(i['input_ids'][0])

In [ ]:
##inference

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

In [ ]:
# Inference
peft_model_id = "/content/gemma_function_calling_and_thinking/checkpoint-400"

model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, torch_dtype=torch.float16, device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

In [ ]:
# from peft import PeftModel

# ft_model = PeftModel.from_pretrained(base_model, "/kaggle/working/peft-dialogue-summary-training-1705417060/checkpoint-1000",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
input = """<start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'search_restaurants', 'description': 'Search for restaurants in a specific location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to search for restaurants'}, 'cuisine': {'type': 'string', 'description': 'The cuisine type to filter the restaurants'}, 'price_range': {'type': 'integer', 'description': 'The price range of the restaurants (1-4)'}}, 'required': ['location']}}}, {'type': 'function', 'function': {'name': 'generate_random_password', 'description': 'Generate a random password', 'parameters': {'type': 'object', 'properties': {'length': {'type': 'integer', 'description': 'The length of the password'}}, 'required': ['length']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call>Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thought}</think>

I'm in New York and I'm craving Italian food. Can you find a restaurant for me?<eos_turn><eos>
<start_of_turn>model"""

In [ ]:
inp = tokenizer.encode(input, return_tensors='pt',add_special_tokens=False).to('cuda')

In [ ]:
print(tokenizer.decode(inp[0]))

In [ ]:
out = model.generate(inp, max_new_tokens=200)

In [ ]:
print(tokenizer.decode(out[0]))

In [ ]:
print(tokenizer.decode(ds['train'][0]['messages']))

In [ ]:
model.push_to_hub("your-username/finetuned-gpt2")
tokenizer.push_to_hub("your-username/finetuned-gpt2")